### Clean and save data
- Remove irrelevant columns
- Remove timezone information (convert to UTC) and convert datetime objects for easier indexing in the environment 
- Only save data from Oct 1, 2021 to Sep 30, 2024 (inclusive  on both ends)

In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from glob import glob

# desired data range
start_date = pd.Timestamp('2021-10-01', tz='UTC')
end_date = pd.Timestamp('2024-10-01', tz='UTC')

def get_desired_dates(df):
    df['interval_start'] = pd.to_datetime(df['interval_start'], utc=True)
    df['interval_end'] = pd.to_datetime(df['interval_end'], utc=True)
    df = df[(df['interval_start'] >= start_date) & (df['interval_start'] < end_date)].reset_index(drop=True)
    return df

Load and load forecast

In [2]:
load_df = pd.read_csv(f'raw_data/load/caiso_5min_load_20210901_20241101')
load_df = load_df[['Interval Start', 'Interval End', 'Load']]
load_df = load_df.rename(columns={"Interval Start": "interval_start", "Interval End": "interval_end", "Load": "load"})
load_df = get_desired_dates(load_df)

load_forecast_df = pd.read_csv(f'raw_data/load_forecast/caiso_hourly_load_forecast_20210901_20241101')
load_forecast_df = load_forecast_df[['Interval Start', 'Interval End', 'Load Forecast']]
load_forecast_df = load_forecast_df.rename(columns={"Interval Start": "interval_start", "Interval End": "interval_end", "Load Forecast": "load_forecast"})
load_forecast_df = get_desired_dates(load_forecast_df)

In [3]:
load_df

,interval_start,interval_end,load
0,2021-10-01 00:00:00+00:00,2021-10-01 00:05:00+00:00,31334.0
1,2021-10-01 00:05:00+00:00,2021-10-01 00:10:00+00:00,31260.0
2,2021-10-01 00:10:00+00:00,2021-10-01 00:15:00+00:00,31326.0
3,2021-10-01 00:15:00+00:00,2021-10-01 00:20:00+00:00,31459.0
4,2021-10-01 00:20:00+00:00,2021-10-01 00:25:00+00:00,31561.0
...,...,...,...
315377,2024-09-30 23:35:00+00:00,2024-09-30 23:40:00+00:00,33413.0
315378,2024-09-30 23:40:00+00:00,2024-09-30 23:45:00+00:00,33642.0
315379,2024-09-30 23:45:00+00:00,2024-09-30 23:50:00+00:00,33803.0
315380,2024-09-30 23:50:00+00:00,2024-09-30 23:55:00+00:00,34017.0


In [4]:
load_forecast_df

,interval_start,interval_end,load_forecast
0,2021-10-01 00:00:00+00:00,2021-10-01 01:00:00+00:00,31787.86
1,2021-10-01 01:00:00+00:00,2021-10-01 02:00:00+00:00,31916.96
2,2021-10-01 02:00:00+00:00,2021-10-01 03:00:00+00:00,31897.77
3,2021-10-01 03:00:00+00:00,2021-10-01 04:00:00+00:00,30692.06
4,2021-10-01 04:00:00+00:00,2021-10-01 05:00:00+00:00,29122.63
...,...,...,...
18859,2024-09-30 19:00:00+00:00,2024-09-30 20:00:00+00:00,24891.31
18860,2024-09-30 20:00:00+00:00,2024-09-30 21:00:00+00:00,26586.11
18861,2024-09-30 21:00:00+00:00,2024-09-30 22:00:00+00:00,28903.05
18862,2024-09-30 22:00:00+00:00,2024-09-30 23:00:00+00:00,31517.29


In [5]:
load_df.to_csv(f'cleaned_data/load/caiso_5min_load_20211001_20240930')
load_forecast_df.to_csv(f'cleaned_data/load_forecast/caiso_hourly_load_forecast_20211001_20240930')

Solar and wind forecast

In [6]:
solar_wind_forecast_df = pd.read_csv(f'raw_data/solar_wind_forecast/caiso_sp15_hourly_solar_wind_forecast_20210901_20241101')
solar_wind_forecast_df = solar_wind_forecast_df[solar_wind_forecast_df['Location']=='SP15'].reset_index(drop=True)
solar_wind_forecast_df = solar_wind_forecast_df[['Interval Start', 'Interval End', 'Solar MW', 'Wind MW']]
solar_wind_forecast_df = solar_wind_forecast_df.rename(columns={"Interval Start": "interval_start", "Interval End": "interval_end", "Solar MW": "solar_mw", "Wind MW": "wind_mw"})
solar_wind_forecast_df = get_desired_dates(solar_wind_forecast_df)

In [7]:
solar_wind_forecast_df

,interval_start,interval_end,solar_mw,wind_mw
0,2021-10-01 00:00:00+00:00,2021-10-01 01:00:00+00:00,2909.65,163.76
1,2021-10-01 01:00:00+00:00,2021-10-01 02:00:00+00:00,245.34,135.17
2,2021-10-01 02:00:00+00:00,2021-10-01 03:00:00+00:00,6.60,183.30
3,2021-10-01 03:00:00+00:00,2021-10-01 04:00:00+00:00,0.00,223.14
4,2021-10-01 04:00:00+00:00,2021-10-01 05:00:00+00:00,0.00,253.84
...,...,...,...,...
18859,2024-09-30 19:00:00+00:00,2024-09-30 20:00:00+00:00,11904.57,370.60
18860,2024-09-30 20:00:00+00:00,2024-09-30 21:00:00+00:00,11854.10,357.28
18861,2024-09-30 21:00:00+00:00,2024-09-30 22:00:00+00:00,11691.08,311.74
18862,2024-09-30 22:00:00+00:00,2024-09-30 23:00:00+00:00,11216.28,239.73


In [8]:
solar_wind_forecast_df.to_csv(f'cleaned_data/solar_wind_forecast/caiso_sp15_hourly_solar_wind_forecast_20211001_20240930')

LMP data

In [9]:
lmp_df = pd.read_csv(f'raw_data/lmp/caiso_5min_rtm_sp15_lmp_20210901_20241101')
lmp_df = lmp_df[['Interval Start', 'Interval End', 'LMP']]
lmp_df = lmp_df.rename(columns={"Interval Start": "interval_start", "Interval End": "interval_end", "LMP": "lmp"})
lmp_df = get_desired_dates(lmp_df)

In [10]:
lmp_df

,interval_start,interval_end,lmp
0,2021-10-01 00:00:00+00:00,2021-10-01 00:05:00+00:00,38.21280
1,2021-10-01 00:05:00+00:00,2021-10-01 00:10:00+00:00,39.37177
2,2021-10-01 00:10:00+00:00,2021-10-01 00:15:00+00:00,41.12420
3,2021-10-01 00:15:00+00:00,2021-10-01 00:20:00+00:00,47.68437
4,2021-10-01 00:20:00+00:00,2021-10-01 00:25:00+00:00,50.93666
...,...,...,...
226363,2024-09-30 23:35:00+00:00,2024-09-30 23:40:00+00:00,31.42568
226364,2024-09-30 23:40:00+00:00,2024-09-30 23:45:00+00:00,37.82225
226365,2024-09-30 23:45:00+00:00,2024-09-30 23:50:00+00:00,49.38882
226366,2024-09-30 23:50:00+00:00,2024-09-30 23:55:00+00:00,57.44217


In [11]:
lmp_df.to_csv(f'cleaned_data/lmp/caiso_5min_rtm_sp15_lmp_20211001_20240930')

MOER

In [12]:
# concatenate all the files into one dataframe
moer_dfs = []
for file in glob(f'raw_data/moer/sdge/*'):
    moer_dfs.append(pd.read_csv(file))
moer_df = pd.concat(moer_dfs).reset_index(drop=True)

# clean the data
moer_df['interval_start'] = pd.to_datetime(moer_df['timestamp'], utc=True)
moer_df['interval_end'] = moer_df['interval_start'] + datetime.timedelta(minutes=5)
moer_df['moer'] = moer_df['MOER version 2.0']
moer_df = moer_df[['interval_start', 'interval_end', 'moer']]
moer_df = get_desired_dates(moer_df)

In [13]:
moer_df

,interval_start,interval_end,moer
0,2021-10-01 00:00:00+00:00,2021-10-01 00:05:00+00:00,0.327031
1,2021-10-01 00:05:00+00:00,2021-10-01 00:10:00+00:00,0.324405
2,2021-10-01 00:10:00+00:00,2021-10-01 00:15:00+00:00,0.327878
3,2021-10-01 00:15:00+00:00,2021-10-01 00:20:00+00:00,0.349154
4,2021-10-01 00:20:00+00:00,2021-10-01 00:25:00+00:00,0.361276
...,...,...,...
315642,2024-09-30 23:35:00+00:00,2024-09-30 23:40:00+00:00,0.307610
315643,2024-09-30 23:40:00+00:00,2024-09-30 23:45:00+00:00,0.444552
315644,2024-09-30 23:45:00+00:00,2024-09-30 23:50:00+00:00,0.521970
315645,2024-09-30 23:50:00+00:00,2024-09-30 23:55:00+00:00,0.656199


In [14]:
moer_df.to_csv(f'cleaned_data/moer/caiso_sdge_5min_moer_v2_20211001_20240930')